#### Imports & constants & tunable parameters

In [53]:
import math
from os import listdir
from re import search

# affinity for susceptibility (negative/positive) - indices and directories
NEG, POS, TEST = 0, 1, 2

# sum of significance threshold parameter
ALPHA = 0.5

# weight function for a mutation given total # mutations in sequence
def weight(n):
  return (1 / (math.log(n) + 2))

# length of human ACE2 sequence
SEQ_LEN = 805

#### Parse sequences from FASTA files

The sequences are divided into:
<ol start="0">
  <li>Known insusceptible sequences</li>
  <li>Known susceptible sequences</li>
  <li>Test sequences for predictions</li>
</ol>

Sequences are stored in tuples as (species name, sequence).

In [54]:
# complete list of data in format (organism, sequence) for each affinity
seqs = [[], [], []]

for aff in [NEG, POS, TEST]:
  for filename in listdir('./data/%d' % aff):
    with open("./data/%d/%s" % (aff, filename)) as f:
      lines = f.readlines()
      name = search(r'\[(.*?)\]',lines[0][1:]).group(1)
      sequence = ''.join(map(lambda k: k.strip(), lines[1:])).strip()
      seqs[aff].append((name, sequence))

# seqs

#### Consensus Sequence

The sequence made up of the most common residues at each site among the susceptible sequences

In [55]:
consensus_seq = ""

for i in range(SEQ_LEN):
  acids = []
  for (name, seq) in seqs[POS]:
    acids.append(seq[i])
  consensus_seq += max(acids, key=acids.count)

consensus_seq

'MSGSFWLLLSLVAVTAAQSTTEELAKTFLEKFNHEAEDLSYQSSLASWNYNTNITDENVQKMNEAGAKWSAFYEEQSKLAKTYPLQEIQNLTVKRQLQALQQSGSSVLSADKSKRLNTILNTMSTIYSTGKVCNPNNPQECLLLEPGLDDIMENSKDYNERLWAWEGWRSEVGKQLRPLYEEYVVLKNEMARANNYEDYGDYWRGDYEVEGVDGYDYSRDQLIEDVERTFTEIKPLYEHLHAYVRAKLMDAYPSYISPTGCLPAHLLGDMWGRFWTNLYSLTVPFGQKPNIDVTDAMVNQSWDARRIFKEAEKFFVSVGLPNMTQGFWENSMLTEPGDGRKVVCHPTAWDLGKGDFRIKMCTKVTMDDFLTAHHEMGHIQYDMAYAAQPFLLRNGANEGFHEAVGEIMSLSAATPNHLKSIGLLSPDFSEDNETEINFLLKQALTIVGTLPFTYMLEKWRWMVFKGEIPKEQWMQKWWEMKREIVGVVEPVPHDETYCDPASLFHVANDYSFIRYYTRTIYQFQFQEALCQIAKHEGPLHKCDISNSTEAGQKLLQMLSLGKSEPWTLALENVVGAKNMDVRPLLNYFEPLFTWLKEQNRNSFVGWSTDWSPYADQSIKVRISLKSALGDKAYEWNDNEMYLFRSSVAYAMREYFLKVKNQTIPFGEEDVRVSDLKPRISFNFFVTAPKNVSDIIPRTEVEEAIRMSRSRINDAFRLDDNSLEFLGIQPTLEPPYQPPVTIWLIVFGVVMGVVVVGIVLLIFTGIRDRRKKNQARSEENPYASVDLSKGENNPGFQNTDDVQTSF'

#### Identifying influential mutations

Find mutations that exist exclusively in insusceptible sequences, assign a weight to each based on the total count, and sum the weights for each negative sequence. The mutations whose weights exceed a defined threshold (alpha) are recorded.

In [56]:

mutts = {}

for (name, seq) in seqs[NEG]:
  n_mutations = 0
  ix_mutation = []
  for (i, ch_i) in enumerate(seq):
    if (ch_i) == "-": continue
    has_match = False
    for (_, seq_pos) in seqs[POS]:
      if (seq_pos[i] == ch_i): 
        has_match = True
        break
    if (not has_match): 
      n_mutations += 1
      mut = "%c%d" % (ch_i, i + 1)
      ix_mutation.append(mut) 
  for i in ix_mutation:
    # computed weighted value = 1 / (log(n) + 2)
    wv = weight(n_mutations)
    mutts[i] = wv if i not in mutts else mutts[i] + wv

result = []
for (m, f) in mutts.items():
  if (f >= ALPHA): 
    m, i = m[0], int(m[1:])
    result.append((i, m, f))
result.sort(key=lambda k: k[0])

result

[(31, 'D', 0.6863135611816127),
 (41, 'A', 0.5),
 (66, 'A', 0.5385357335328163),
 (83, 'F', 0.5226018801516173),
 (113, 'N', 0.525115887738091),
 (353, 'H', 0.8362883189700047),
 (426, 'S', 0.5226018801516173),
 (679, 'V', 0.525115887738091)]

#### Important sites

Print residues at influential sites for susceptible (including the consensus sequence) and then insusceptible sequences.

In [57]:
indices = list(map(lambda k: k[0], result))

def print_influence_pts(seq, name, aff):
  p_str = "  "
  for i in indices: p_str += seq[i-1] + "   "
  p_str += "(%s) %s" % (aff, name)
  print(p_str)

print(''.join(["%4d" % (i) for i in indices]))
for (name, seq) in seqs[POS]: print_influence_pts(seq, name, "+") 
print_influence_pts(consensus_seq, "Consensus sequence", "+")
print()
for (name, seq) in seqs[NEG]: print_influence_pts(seq, name, "-")

  31  41  66  83 113 353 426 679
  K   Y   G   Y   S   K   P   I   (+) Felis catus
  K   Y   G   Y   S   K   P   I   (+) Chlorocebus sabaeus
  K   Y   G   Y   S   K   P   I   (+) Rhinolophus macrotis
  K   Y   E   Y   S   K   Y   I   (+) Oryctolagus cuniculus
  K   H   G   Y   S   K   P   I   (+) Callithrix jacchus
  K   Y   G   Y   S   K   P   I   (+) Macaca fascicularis
  K   Y   R   Y   S   K   P   -   (+) Odocoileus virginianus texanus
  K   Y   G   Y   S   K   P   I   (+) Panthera tigris
  K   Y   G   Y   R   K   P   I   (+) Mustela putorius furo
  K   Y   R   H   T   K   P   I   (+) Cynopterus sphinx
  K   Y   G   Y   R   K   P   I   (+) Mustela lutreola biedermanni
  K   Y   G   Y   S   K   P   I   (+) Homo sapiens
  K   Y   G   Y   S   K   P   I   (+) Consensus sequence

  -   A   -   -   -   -   -   -   (-) Mutation A41
  N   Y   A   Y   R   K   P   I   (-) Procyon lotor
  D   Y   -   -   -   -   -   -   (-) Mutation D31
  -   -   -   -   -   H   -   -   (-) Mutation H353
  N 

#### Predictions based on results

Predict the affinity of test sequences based on the presence or absence of influential mutations

In [58]:
def predict(seq):
  for (i, m, _) in result:
    if seq[i] == m: return "-"
  return "+"

print(''.join(["%4d" % (i) for i in indices]))
for (name, seq) in seqs[TEST]: print_influence_pts(seq, name, predict(seq)) 
print()


  31  41  66  83 113 353 426 679
  F   E   A   S   S   K   P   V   (+) Meleagris gallopavo
  K   Y   R   Y   S   K   P   I   (-) Capra hircus
  K   Y   G   F   S   K   P   I   (+) Mutation NFS82
  -   -   -   -   -   K   P   I   (+) Columba livia
  K   Y   A   Y   N   K   S   V   (-) Mesocricetus auratus
  K   Y   G   Y   N   R   P   I   (-) Nyctereutes procyonoides
  F   E   A   S   S   K   P   V   (+) Coturnix japonica
  F   E   A   S   S   K   P   V   (+) Gallus gallus

